In [10]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [11]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [12]:
def write_json_to_txt(json_data, file_name):
    filtered_data = {k: v for k, v in json_data.items() if v.get("reflect") == "yes"}
    with open(file_name, 'w') as file:
        for test_case, details in filtered_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [13]:
SYSTEM_PROMPT_8="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.

Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned."
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""

In [14]:
SELF_REF="""
Given a test scenario and test cases to test that given test scenario.
Mark if test case can test the given test scenario or not through test steps, expected output, objective (although if it test other use case path or flow, if it is not used to test the given test scenario.)
And give explanation why you think the resulted test case reflect the given test scenario or not. 
Your response should keep the format of the inserted test cases.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "reflect": "yes/no",
    "explanation": "explain why you think this test case reflect the given test scenario or not",
  ],
}"""

In [15]:
scenarios = [
"1. Display First Page of Authors",
"2. Navigate to Next Page of Authors",
"3. Navigate to a Specific Page of Authors",
"4. Last Page of Authors Displayed"]

In [16]:
prompt_uc = """
Display the list of authors
Description
The page is designed to list all the authors.

Main scenario:
User selects item "Authors";
Application displays the list of the authors (up to 10 items on each page);
To see the next 10 records of the authors in the database, the user needs to click the "Next" button (below the records in the pagination block) or the following number of the page the user wants to see.

The list displays the following columns:
id - unique author number
Name
Bio - author's biography
"""

In [17]:
for scenario in scenarios:
    i = 1
    while(i<=1):
        try: 
            promptTestCase = [
            { "role": "system", "content": SYSTEM_PROMPT_8},
            { "role": "user", "content": scenario + "\n" + prompt_uc}
            ]
            gpt_response = ask(promptTestCase, client, model)
            json_data = json.loads(gpt_response)
            write_json_to_txt_raw(json_data, scenario + "-Raw" + str(i) + ".txt")

            promptSelfRef = [
                { "role": "system", "content": SELF_REF},
                { "role": "user", "content": scenario + "\n" + gpt_response}
            ]
            final_response = ask(promptSelfRef, client, model)
            json_fin = json.loads(final_response)
            write_json_to_txt(json_fin, scenario + str(i) + ".txt")
            i+=1
        except Exception as e:
            print(f"An error occurred with scenario '{scenario}' iteration {i}: {e}")